In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U adapter-transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("imdb",split='train')
test_dataset_sun = load_dataset("imdb",split='test[:100]')
test_dataset = load_dataset("imdb",split='test')

# train_dataset = load_dataset("yelp_polarity",split='train')
# test_dataset_sun = load_dataset("yelp_polarity",split='test[:100]')
# test_dataset = load_dataset("yelp_polarity",split='test')

# dataset = load_dataset("imdb")


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


In [ ]:
train_dataset.num_rows

25000

In [ ]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
test_dataset.num_rows

25000

In [ ]:
test_dataset_sun.num_rows

100

In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def encode_batch(batch):
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

train_dataset = train_dataset.map(encode_batch, batched=True)
train_dataset.set_format(type="torch", columns=['text',"input_ids", "attention_mask", "label"])

test_dataset = test_dataset.map(encode_batch, batched=True)
test_dataset.set_format(type="torch", columns=['text',"input_ids", "attention_mask", "label"])

test_dataset_sun = test_dataset_sun.map(encode_batch, batched=True)
test_dataset_sun.set_format(type="torch", columns=['text',"input_ids", "attention_mask", "label"])

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_dataset['label']

tensor([0, 0, 0,  ..., 1, 1, 1])

In [ ]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction, Trainer
from transformers.adapters import PrefixTuningConfig
from transformers import  BertModel, AutoModelForSequenceClassification




def compute_accuracy(p: EvalPrediction):
      preds = np.argmax(p.predictions, axis=1)
      return {"acc": (preds == p.label_ids).mean()}

def training(type_model):
  config = AutoConfig.from_pretrained(
      "bert-base-uncased",
      num_labels=2,
  )
  model = AutoModelForSequenceClassification.from_pretrained(
      "bert-base-uncased",
      config=config,
  )
  if(type_model==1):
    config = PrefixTuningConfig(flat=False, prefix_length=30)
    model.add_adapter("prefix_tuning", config=config)
    model.train_adapter("prefix_tuning")

    training_args = TrainingArguments(
        learning_rate=5e-5,
        num_train_epochs=5,
        per_device_train_batch_size=128,
        per_device_eval_batch_size=128,
        logging_steps=200,
        output_dir="./training_output",
        overwrite_output_dir=True,
        # The next line is important to ensure the dataset labels are properly passed to the model
        remove_unused_columns=False,
    )

    

    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_accuracy,
    )

  else:
    training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,)



    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_accuracy,
    )
  return trainer

In [ ]:
trainer = training(0)
trainer.train()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/re

Step,Training Loss
200,0.417800
400,0.244500
600,0.110300
800,0.051500
1000,0.030900
1200,0.025500
1400,0.011300
1600,0.010200
1800,0.004900


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/config.json
Model weights saved in ./training_output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./training_output/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./training_output/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/config.json
Model weights saved in ./training_output/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./training_output/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./training_output/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./training_output/checkpoint-1500
Configuration saved in ./training_output/checkpoint-1500/config.json
Model weights saved in ./training_output/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./training_

TrainOutput(global_step=1960, training_loss=0.0930555204043583, metrics={'train_runtime': 3346.7748, 'train_samples_per_second': 74.699, 'train_steps_per_second': 0.586, 'total_flos': 1.02777756e+16, 'train_loss': 0.0930555204043583, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 100
  Batch size = 128


{'eval_loss': 0.5337377786636353,
 'eval_acc': 0.77,
 'eval_runtime': 0.6425,
 'eval_samples_per_second': 155.643,
 'eval_steps_per_second': 1.556,
 'epoch': 5.0}

In [ ]:
base = "/content/drive/MyDrive/Capstone/"

In [ ]:
test_dataset.save_to_disk(base+"test.hf")

In [ ]:
from datasets import load_from_disk

temp = load_from_disk(base+"test.hf")

temp

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})

In [ ]:
# from transformers import TextClassificationPipeline

# classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

# classifier("This is awesome!")

In [ ]:
trainer.save_model(base+"finetune_yelp")

Saving model checkpoint to /content/drive/MyDrive/Capstone/adapter_imdb
Configuration saved in /content/drive/MyDrive/Capstone/adapter_imdb/prefix_tuning/adapter_config.json
Module weights saved in /content/drive/MyDrive/Capstone/adapter_imdb/prefix_tuning/pytorch_adapter.bin
Configuration saved in /content/drive/MyDrive/Capstone/adapter_imdb/prefix_tuning/head_config.json
Module weights saved in /content/drive/MyDrive/Capstone/adapter_imdb/prefix_tuning/pytorch_model_head.bin


In [ ]:
model.save_adapter("./final_adapter", "rotten_tomatoes")

# !ls -lh final_adapter